In [1]:
import sys
sys.path.append('../')
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import json
import urllib
import time 
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
from glob import glob
from collections import ChainMap
from tqdm.notebook import tqdm as tqdm_notebook
from datetime import datetime 
from config import chromeSetup,get_selected_cols_df,ensure_dir,get_Drivers,encode_search_word,enter_sys_value,update_iod_tms_with_now,transform

In [2]:

from config import Marco

marco = Marco()
marco.colsSend2Carrier

['Sales',
 'Load ID',
 'Stop Seq',
 'Carrier Code',
 'Carrier Name',
 'Service Type',
 'Alert Type',
 'Ship Date',
 'ETA',
 'From Address',
 'To Address',
 'Customer Name',
 'GERP Order No',
 'Pick Order No',
 'Pro No']

In [3]:




CARRIER_CODE = ['RDWY_CRA',
                 'RDWY_FON',
                 'RDWY_FTW',
                 'RDWY_POR',
                 'RDWY_ROM',
                'RDWY'
               ]
FILE_PATH = glob(".\data\*.csv")[-1]
df_origin = pd.read_csv(FILE_PATH)
df = df_origin[df_origin['Carrier Code'].isin(CARRIER_CODE)]



from selenium import webdriver
import time 

chrome_options = Options()
chrome_options.add_argument("--headless")
browers = webdriver.Chrome("C:\\Users\\marco.wang\\Downloads\\chromedriver_win32\\chromedriver",options=chrome_options)
 
    
def check(load):
    try:
        int(load)
        return True
    except:
        return False

def clean(text):
    return "".join([ i for i in text if i.isdigit() or i == "X"])

result = []

In [4]:

loads = df["Pro No"].to_list()

for load in tqdm_notebook(loads):

    try:
        url = f'https://my.yrc.com/tools/track/shipments?referenceNumber={load}&referenceNumberType=PRO&time={int(time.mktime(datetime.now().timetuple()))}'
        browers.get(url)
        time.sleep(2)
        t = browers.find_elements_by_class_name("panel-body")

        if len(t) > 1:
            d = t[1].text.split("\n")
            data = {}
            for k in range(0,len(d),2):
                if k == len(d): continue
                data[d[k]] = d[k+1]

            data['YRC Freight PRO #:'] = clean(data['YRC Freight PRO #:'] )
            result.append({
                data['YRC Freight PRO #:']:data 
            })
    except:
        print(load)


In [11]:
result = dict(ChainMap(*result))
print(len(result))

timestamp = str(datetime.now())  

df = df[marco.colsSend2Carrier]

today_date = timestamp[:10]
timestamp = "".join([d for d  in timestamp if d.isdigit()])
ensure_dir(os.path.join("table_result",today_date))
df['Status'] = df['Pro No'].apply(lambda x : result[x]['Status:'] if result.get(x) and result.get(x).get("Status:")  else "null")
df['Status Updated:'] = df['Pro No'].apply(lambda x : result[x]['Status Updated:'] if result.get(x) and result.get(x).get('Status Updated:') else "null")
df['Delivered Date'] = df['Pro No'].apply(lambda x : result[x]['Delivered Date'] if result.get(x) and result.get(x).get('Delivered Date') else "null" )
df['Est. Delivery Date:'] = df['Pro No'].apply(lambda x : result[x]['Est. Delivery Date:'] if result.get(x) and result.get(x).get('Est. Delivery Date:') else "null" )
df['Appointment Date:'] = df['Pro No'].apply(lambda x : result[x]['Appointment Date:'] if result.get(x) and result.get(x).get('Appointment Date:') else "null" )




TypeError: string indices must be integers

In [13]:
df.to_excel(os.path.join("table_result",today_date,f"report_RDWY_{timestamp[8:12]}.xlsx"),index=False,engine='xlsxwriter')